- **Import libraries**

In [80]:
import os
from moviepy import VideoFileClip
import cv2
import pandas as pd

In [81]:
def timestamp_to_seconds(timestamp, fps=30):
    hh, mm, ss, ff = map(int, timestamp.split(':'))
    total_seconds = hh * 3600 + mm * 60 + ss + ff / fps
    return total_seconds

- **Function for extracting screenshots in certain time**

In [99]:
def get_screenshots_and_processed_df(video_id):
    def extract_screenshots(video_path, timestamps, output_folder):
        # Check if the output folder exists, if not, create it
        if not os.path.exists(output_folder):
            os.makedirs(output_folder)
        
        video = VideoFileClip(video_path)
        screenshot_paths = []

        # Iterate through timestamps
        for index, timestamp in enumerate(timestamps):
            # Set the video to the specified timestamp
            try:
                timestamp_float = timestamp_to_seconds(timestamp)
                frame = video.get_frame(timestamp_float)  # Ensure timestamp is a float
            except Exception as e:
                print(f"Error getting frame at timestamp {timestamp} : {e}")
                screenshot_paths.append(None)
                continue  # Skip to the next timestamp if there's an error

            # Convert the frame to BGR format (OpenCV uses BGR)
            frame_bgr = cv2.cvtColor(frame, cv2.COLOR_RGB2BGR)
            print(f"Frame Shape: {frame_bgr.shape}, Type: {frame_bgr.dtype}")

            # Save the screenshot
            output_path = f"{output_folder}/screenshot_{index+1}.png"
            success = cv2.imwrite(output_path, frame_bgr)
            if success:
                print(f"Screenshot saved: {output_path}")
                screenshot_paths.append(output_path)
            else:
                print(f"Failed to save screenshot: {output_path}")
                screenshot_paths.append(None)

        # Close the video file
        video.close()
        return screenshot_paths

    def process_num(n):
        return '0' + str(n) if n < 10 else str(n)

    def process_timestamps(timestamps):
        res = []
        for ts in timestamps: 
            ts = ts.split(' ')
            first = int(ts[0][-2:])
            second = int(ts[1][-2:])
            mid = round(abs(int(first) - int(second)) / 2)
            millisec = process_num(first+mid)
            res.append(ts[0][:-2] + millisec)
        return res

    # Load your dataset (assuming it's a CSV file with 'second' and 'video_link' columns)
    df = pd.read_csv(f'./data/{video_id}/transcript_{video_id}.csv')

    # Remove duplicate columns
    df = df.loc[:, ~df.columns.duplicated()]

    # Extract timestamps and video link
    timestamps_unprocessed = df['Zeit'].tolist()[1:]
    video_link = df['video_src'].iloc[0]  # Assuming all rows have the same video link

    # Specify the output folder for screenshots
    output_folder = f'./data/{video_id}/screenshots'

    processed_timestamps = process_timestamps(timestamps_unprocessed)
    
    # Ensure the lengths match
    if len(processed_timestamps) != len(timestamps_unprocessed):
        raise ValueError("Processed timestamps length does not match unprocessed timestamps length")

    screenshot_paths = extract_screenshots(video_link, processed_timestamps, output_folder)

    # Ensure the lengths match
    if len(screenshot_paths) != len(df) - 1:  # Adjust for the header row
        # Adjust the length of screenshot_paths to match the DataFrame
        screenshot_paths += [None] * ((len(df) - 1) - len(screenshot_paths))

    # Generate a DataFrame with the required information
    df_processed = df.iloc[1:][['Zeit', 'Übersetzung', 'Lexem/Gebärde', 'Lexem/Gebärde.1', 'Mund', 'video_src', 'video_id']].copy()
    df_processed['processed_timestamps'] = processed_timestamps
    df_processed['screenshot_path'] = screenshot_paths

    # Group by 'Übersetzung'
    df_grouped = df_processed.groupby('Übersetzung').agg({
        'Zeit': list,
        'Lexem/Gebärde': list,
        'Lexem/Gebärde.1': list,
        'Mund': list,
        'video_src': 'first',
        'video_id': 'first',
        'processed_timestamps': list,
        'screenshot_path': list
    }).reset_index()

    return df_grouped

In [100]:
video_id = '1176340'
df_grouped = get_screenshots_and_processed_df(video_id)

{'video_found': True, 'audio_found': False, 'metadata': {'major_brand': 'isom', 'minor_version': '512', 'compatible_brands': 'isomiso2avc1mp41', 'encoder': 'Lavf59.27.100', 'copyright': '2010-2023 DGS-Korpus, Universität Hamburg'}, 'inputs': [{'streams': [{'input_number': 0, 'stream_number': 0, 'stream_type': 'video', 'language': None, 'default': True, 'size': [1280, 360], 'bitrate': 574, 'fps': 50.0, 'codec_name': 'h264', 'profile': '(Constrained Baseline)', 'metadata': {'Metadata': '', 'handler_name': 'VideoHandler', 'vendor_id': '[0][0][0][0]', 'encoder': 'Lavc59.37.100 libx264'}}], 'input_number': 0}], 'duration': 718.74, 'bitrate': 576, 'start': 0.0, 'default_video_input_number': 0, 'default_video_stream_number': 0, 'video_codec_name': 'h264', 'video_profile': '(Constrained Baseline)', 'video_size': [1280, 360], 'video_bitrate': 574, 'video_fps': 50.0, 'video_duration': 718.74, 'video_n_frames': 35937}
c:\Users\javie\Desktop\ABERDEEN\RESEARCH\sign-language-experiment\venv\Lib\site

c:\Users\javie\Desktop\ABERDEEN\RESEARCH\sign-language-experiment\venv\Lib\site-packages\moviepy\video\io\ffmpeg_reader.py:178: UserWarning: In file https://www.sign-lang.uni-hamburg.de/meinedgs_r4/videos/1176340/1176340.mp4, 1382400 bytes wanted but 0 bytes read at frame index 35948 (out of a total 35937 frames), at time 718.96/718.74 sec. Using the last valid frame instead.
  warnings.warn(


Frame Shape: (360, 1280, 3), Type: uint8
Screenshot saved: ./data/1176340/screenshots/screenshot_3563.png


c:\Users\javie\Desktop\ABERDEEN\RESEARCH\sign-language-experiment\venv\Lib\site-packages\moviepy\video\io\ffmpeg_reader.py:178: UserWarning: In file https://www.sign-lang.uni-hamburg.de/meinedgs_r4/videos/1176340/1176340.mp4, 1382400 bytes wanted but 0 bytes read at frame index 35936 (out of a total 35937 frames), at time 718.72/718.74 sec. Using the last valid frame instead.
  warnings.warn(


In [101]:
df_grouped.to_json(f'./data/{video_id}/processed_transcript_{video_id}.json', index=False)